In [29]:
from pyspark import SparkContext

In [30]:
sc = SparkContext('spark://MILES-BL-4819-LAP.:7077','Walmart Wsl Sales')

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=Walmart Wsl Sales, master=spark://MILES-BL-4819-LAP.:7077) created by __init__ at /tmp/ipykernel_4361/3658538166.py:1 

In [37]:
rdd1=sc.textFile("hdfs://localhost:9000/user/shashi/WalmartSales.csv",1)

In [38]:
head =rdd1.first()
rdd2 = rdd1.filter(lambda row:row != head) #filtering records without header

In [39]:
rdd3 = rdd2.map(lambda x: x.split(','))
rdd3.cache()

PythonRDD[43] at RDD at PythonRDD.scala:53

In [40]:
rdd3.take(1)

[['Central America and the Caribbean',
  'Antigua and Barbuda ',
  'Baby Food',
  'Online',
  'M',
  '12/20/2013',
  '957081544',
  '1/11/2014',
  '552',
  '255.28',
  '159.42',
  '140914.56',
  '87999.84',
  '52914.72']]

In [41]:
# 1. Display the number of countries present in the data(Using Hive)

rdd3.map(lambda x: x[1]).distinct().count()

185

In [42]:
# 2.Display the number of units sold in each region.(Using Hive)

rdd3.map(lambda x: (x[0],int(x[8]))).reduceByKey(lambda x,y : x+y).saveAsTextFile("hdfs://localhost:9000/user/shashi/output/Q2")



In [44]:
# 3.Display the 10 most recent sales. (Using Hive)
from datetime import datetime

rdd4= rdd3.map(lambda row: row[:5] + [datetime.strptime(row[5],'%m/%d/%Y')] + row[6:])
rdd4.sortBy(lambda row : row[5],ascending=False).map(lambda row : row[:5] + [row[5].strftime('%-m/%-d/%Y')] +row[6:]).saveAsTextFile("hdfs://localhost:9000/user/shashi/output/Q3")
rdd4.sortBy(lambda row : row[5],ascending=False).map(lambda row : row[:5] + [row[5].strftime('%-m/%-d/%Y')] +row[6:]).take(10)

[['Asia',
  'Bhutan',
  'Cereal',
  'Offline',
  'M',
  '7/28/2017',
  '223854434',
  '8/25/2017',
  '2356',
  '205.70',
  '117.11',
  '484629.20',
  '275911.16',
  '208718.04'],
 ['Sub-Saharan Africa',
  'Senegal',
  'Cosmetics',
  'Online',
  'C',
  '7/26/2017',
  '537970721',
  '8/18/2017',
  '6346',
  '437.20',
  '263.33',
  '2774471.20',
  '1671092.18',
  '1103379.02'],
 ['Middle East and North Africa',
  'United Arab Emirates',
  'Household',
  'Online',
  'C',
  '7/26/2017',
  '419542396',
  '8/8/2017',
  '773',
  '668.27',
  '502.54',
  '516572.71',
  '388463.42',
  '128109.29'],
 ['Australia and Oceania',
  'Australia',
  'Beverages',
  'Online',
  'L',
  '7/26/2017',
  '631485402',
  '8/12/2017',
  '9418',
  '47.45',
  '31.79',
  '446884.10',
  '299398.22',
  '147485.88'],
 ['Sub-Saharan Africa',
  "Cote d'Ivoire",
  'Vegetables',
  'Online',
  'H',
  '7/24/2017',
  '588388097',
  '8/25/2017',
  '5968',
  '154.06',
  '90.93',
  '919430.08',
  '542670.24',
  '376759.84'],
 ['S

In [45]:
# 4.Display the products with atleast 2 occurences of 'a' (Using spark)
rdd3.filter(lambda x: x[2].count('a')>=2).saveAsTextFile("hdfs://localhost:9000/user/shashi/output/Q4")
rdd3.filter(lambda x: x[2].count('a')>=2).collect()

[['Middle East and North Africa',
  'Morocco',
  'Personal Care',
  'Offline',
  'L',
  '11/8/2010',
  '412882792',
  '11/22/2010',
  '48',
  '81.73',
  '56.67',
  '3923.04',
  '2720.16',
  '1202.88'],
 ['Sub-Saharan Africa',
  'Benin',
  'Personal Care',
  'Online',
  'L',
  '8/7/2016',
  '440603101',
  '9/15/2016',
  '3104',
  '81.73',
  '56.67',
  '253689.92',
  '175903.68',
  '77786.24'],
 ['Middle East and North Africa',
  'Saudi Arabia',
  'Personal Care',
  'Offline',
  'L',
  '2/2/2014',
  '688270556',
  '3/4/2014',
  '3407',
  '81.73',
  '56.67',
  '278454.11',
  '193074.69',
  '85379.42'],
 ['Sub-Saharan Africa',
  'Central African Republic',
  'Personal Care',
  'Offline',
  'H',
  '12/16/2014',
  '863838946',
  '1/5/2015',
  '7995',
  '81.73',
  '56.67',
  '653431.35',
  '453076.65',
  '200354.70'],
 ['Australia and Oceania',
  'Tuvalu',
  'Personal Care',
  'Offline',
  'M',
  '12/21/2016',
  '963251912',
  '2/1/2017',
  '6501',
  '81.73',
  '56.67',
  '531326.73',
  '3684

In [91]:
# 5.Display country in each region with highest units sold. (Using spark)

rdd3.map(lambda x: ((x[0],x[1]),int(x[8]))).reduceByKey(lambda x,y: x+y )\
    .map(lambda x: (x[0][0],(x[0][1],x[1])))\
    .reduceByKey(lambda a,b : max(a,b ,key=lambda x : x[1])).collect()


[('Central America and the Caribbean', ('Grenada', 205943)),
 ('Australia and Oceania', ('Australia', 183909)),
 ('Middle East and North Africa', ('Somalia', 193065)),
 ('Sub-Saharan Africa', ('Equatorial Guinea', 197767)),
 ('Europe', ('Macedonia', 203078)),
 ('Asia', ('Myanmar', 199967)),
 ('North America', ('United States of America', 159519))]

In [86]:
# 6.Display the unit price and unit cost of each item in ascending order. (Using spark)
rdd3.map(lambda x: (x[2],x[9],x[10])).distinct().collect()

[('Baby Food', '255.28', '159.42'),
 ('Snacks', '152.58', '97.44'),
 ('Cereal', '205.70', '117.11'),
 ('Clothes', '109.28', '35.84'),
 ('Cosmetics', '437.20', '263.33'),
 ('Household', '668.27', '502.54'),
 ('Personal Care', '81.73', '56.67'),
 ('Meat', '421.89', '364.69'),
 ('Fruits', '9.33', '6.92'),
 ('Beverages', '47.45', '31.79'),
 ('Office Supplies', '651.21', '524.96'),
 ('Vegetables', '154.06', '90.93')]

In [69]:
# 7.Display the number of sales yearwise. (Using pyspark)
rdd3.map(lambda x: (int(x[5].split('/')[2]),int(x[8]))).reduceByKey(lambda x,y: x+y).sortBy(lambda x: x[1], ascending=False).collect()

[(2015, 3506548),
 (2012, 3485045),
 (2013, 3358584),
 (2011, 3352394),
 (2016, 3280818),
 (2014, 3214899),
 (2010, 3130137),
 (2017, 1825066)]

In [73]:
# 8.Display the number of orders for each item. (Using pyspark)

rdd3.map(lambda x: (x[2],int(x[8]))).reduceByKey(lambda x,y:x+y).sortBy(lambda x: x[1], ascending=False).collect()

[('Fruits', 2341083),
 ('Baby Food', 2274921),
 ('Beverages', 2208169),
 ('Cosmetics', 2141909),
 ('Personal Care', 2134895),
 ('Office Supplies', 2107628),
 ('Household', 2099238),
 ('Vegetables', 2090330),
 ('Clothes', 1983445),
 ('Cereal', 1949601),
 ('Snacks', 1921075),
 ('Meat', 1901197)]

In [ ]:
# 9.Display the countr.with highest calallicing pyspark)
